In [54]:
import pyspark
from pyspark.sql import SparkSession, types
import pandas as pd

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('homeworkWeek5') \
    .getOrCreate()

22/02/25 10:34:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [8]:
df = spark.read.csv('./data/fhvhv/fhvhv_tripdata_2021-02.csv', header=True)

In [9]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

In [64]:
schema = types.StructType([
    types.StructField('hvfhs_license_num',types.StringType(), True),
    types.StructField('dispatching_base_num',types.StringType(),True),
    types.StructField('pickup_datetime',types.TimestampType(),True),
    types.StructField('dropoff_datetime',types.TimestampType(),True),
    types.StructField('PULocationID',types.StringType(),True),
    types.StructField('DOLocationID',types.StringType(),True),
    types.StructField('SR_Flag',types.StringType(),True)])

In [14]:
df = spark.read.option("header", "true").schema(schema).csv('./data/fhvhv/fhvhv_tripdata_2021-02.csv')

In [17]:
df.head(10)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 2, 1, 0, 10, 40), dropoff_datetime=datetime.datetime(2021, 2, 1, 0, 21, 9), PULocationID='35', DOLocationID='39', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 2, 1, 0, 27, 23), dropoff_datetime=datetime.datetime(2021, 2, 1, 0, 44, 1), PULocationID='39', DOLocationID='35', SR_Flag=None),
 Row(hvfhs_license_num='HV0005', dispatching_base_num='B02510', pickup_datetime=datetime.datetime(2021, 2, 1, 0, 28, 38), dropoff_datetime=datetime.datetime(2021, 2, 1, 0, 38, 27), PULocationID='39', DOLocationID='91', SR_Flag=None),
 Row(hvfhs_license_num='HV0005', dispatching_base_num='B02510', pickup_datetime=datetime.datetime(2021, 2, 1, 0, 43, 37), dropoff_datetime=datetime.datetime(2021, 2, 1, 1, 23, 20), PULocationID='91', DOLocationID='228', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02872', p

In [19]:
df = df.repartition(24)

In [20]:
df.write.parquet('./data/fhvhv/pq/')

22/02/25 21:23:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/02/25 21:23:38 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/02/25 21:23:41 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [21]:
df = spark.read.parquet('./data/fhvhv/pq/')

In [22]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [23]:
df.count().

11613942

In [24]:
df.registerTempTable('fhvhv')

In [34]:
spark.sql("""
select count(*) from fhvhv where pickup_datetime between '2021-02-15 00:00:00' and '2021-02-15 23:59:59'
""").show()

+--------+
|count(1)|
+--------+
|  367170|
+--------+



In [47]:
spark.sql("""
select pickup_datetime, dropoff_datetime, (unix_timestamp(dropoff_datetime)-unix_timestamp(pickup_datetime))/(60) as diff from fhvhv order by diff desc
""").show()

+-------------------+-------------------+------------------+
|    pickup_datetime|   dropoff_datetime|              diff|
+-------------------+-------------------+------------------+
|2021-02-11 13:40:44|2021-02-12 10:39:44|            1259.0|
|2021-02-17 15:54:53|2021-02-18 07:48:34| 953.6833333333333|
|2021-02-20 12:08:15|2021-02-21 00:22:14| 733.9833333333333|
|2021-02-03 20:24:25|2021-02-04 07:41:58|            677.55|
|2021-02-19 23:17:44|2021-02-20 09:44:01| 626.2833333333333|
|2021-02-25 17:13:35|2021-02-26 02:57:05|             583.5|
|2021-02-20 01:36:13|2021-02-20 11:16:19|             580.1|
|2021-02-18 15:24:19|2021-02-19 01:01:11| 576.8666666666667|
|2021-02-18 01:31:20|2021-02-18 11:07:15| 575.9166666666666|
|2021-02-10 20:51:39|2021-02-11 06:21:08| 569.4833333333333|
|2021-02-10 01:56:17|2021-02-10 10:57:33| 541.2666666666667|
|2021-02-25 09:18:18|2021-02-25 18:18:57|            540.65|
|2021-02-21 19:59:13|2021-02-22 04:56:16|            537.05|
|2021-02-09 18:36:13|202

In [51]:
spark.sql("""
select dispatching_base_num, count(dispatching_base_num) as count
from fhvhv group by dispatching_base_num order by count desc """).show()

+--------------------+-------+
|dispatching_base_num|  count|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
|              B02869| 429720|
|              B02887| 322331|
|              B02871| 312364|
|              B02864| 311603|
|              B02866| 311089|
|              B02878| 305185|
|              B02682| 303255|
|              B02617| 274510|
|              B02883| 251617|
|              B02884| 244963|
|              B02882| 232173|
|              B02876| 215693|
|              B02879| 210137|
|              B02867| 200530|
|              B02877| 198938|
+--------------------+-------+
only showing top 20 rows



Number of stages taken is 2

In [53]:
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

--2022-02-25 22:55:38--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.109.134
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.109.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘taxi+_zone_lookup.csv’

taxi+_zone_lookup.c 100%[===================>]  12.03K  --.-KB/s    in 0s      

2022-02-25 22:55:39 (84.3 MB/s) - ‘taxi+_zone_lookup.csv’ saved [12322/12322]



In [57]:
taxi_df = spark.read.csv('taxi+_zone_lookup.csv', header=True)

In [60]:
taxi_df.schema

StructType(List(StructField(LocationID,StringType,true),StructField(Borough,StringType,true),StructField(Zone,StringType,true),StructField(service_zone,StringType,true)))

In [65]:
taxi_schema = types.StructType([
        types.StructField('LocationID',types.StringType(),True),
        types.StructField('Borough',types.StringType(),True),
        types.StructField('Zone',types.StringType(),True),
        types.StructField('service_zone',types.StringType(),True)])

In [66]:
taxi_df = spark.read.option('head', True).schema(taxi_schema).csv('taxi+_zone_lookup.csv')

In [69]:
taxi_df.registerTempTable('taxi_zone')

In [71]:
spark.sql("""
select * from taxi_zone""").show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhatta

In [80]:
spark.sql("""

SELECT t1.Zone as pickup_zone, CASE WHEN t2.Zone is NULL THEN 'Unknown' ELSE t2.Zone END AS destination_zone  
from fhvhv y INNER JOIN taxi_zone t1 on t1.LocationID= y.PULocationID inner join 
taxi_zone t2 on t2.LocationID=y.DOLocationID
GROUP BY 1,2 
--order by avg(total_amount) DESC LIMIT 1

""").show()

+--------------------+--------------------+
|         pickup_zone|    destination_zone|
+--------------------+--------------------+
|              Inwood| Kingsbridge Heights|
|     Jackson Heights|  Murray Hill-Queens|
|           Flatlands|Marine Park/Mill ...|
|           Bronxdale|         Eastchester|
|      Queens Village|                  NA|
|      Middle Village|      Middle Village|
|           Sunnyside|         Old Astoria|
|           Gravesend|      Sheepshead Bay|
|Schuylerville/Edg...|Williamsbridge/Ol...|
|Mott Haven/Port M...|       Melrose South|
|Prospect-Lefferts...|Downtown Brooklyn...|
|           Homecrest|    Bensonhurst East|
|University Height...|West Farms/Bronx ...|
|          Co-Op City|        West Village|
|          Park Slope|     Jackson Heights|
|Marine Park/Mill ...| Crown Heights South|
|TriBeCa/Civic Center|    Bensonhurst East|
|        Far Rockaway|         Hunts Point|
|    Prospect Heights|Downtown Brooklyn...|
|East Concourse/Co...|        Be

In [103]:
spark.sql("""

select from_to, count(from_to) as count from
(select concat(pickup_zone, '/', destination_zone) as from_to from (
SELECT t1.Zone as pickup_zone, CASE WHEN t2.Zone is NULL THEN 'Unknown' ELSE t2.Zone END AS destination_zone
from fhvhv y INNER JOIN taxi_zone t1 on t1.LocationID= y.PULocationID inner join 
taxi_zone t2 on t2.LocationID=y.DOLocationID))
group by 1 order by count desc
""").show(truncate=False)

+---------------------------------------------------+-----+
|from_to                                            |count|
+---------------------------------------------------+-----+
|East New York/East New York                        |45041|
|Borough Park/Borough Park                          |37329|
|Canarsie/Canarsie                                  |28026|
|Crown Heights North/Crown Heights North            |25976|
|Bay Ridge/Bay Ridge                                |17934|
|Jackson Heights/Jackson Heights                    |14688|
|Astoria/Astoria                                    |14688|
|Central Harlem North/Central Harlem North          |14481|
|Bushwick South/Bushwick South                      |14424|
|Flatbush/Ditmas Park/Flatbush/Ditmas Park          |13976|
|South Ozone Park/South Ozone Park                  |13716|
|Brownsville/Brownsville                            |12829|
|JFK Airport/NA                                     |12542|
|Prospect-Lefferts Gardens/Crown Heights

In [104]:
!ls -lh /home/saheedyusuf/spark/week_5_batch_processing/code/data/fhvhv/pq

total 206M
-rw-r--r-- 1 saheedyusuf saheedyusuf    0 Feb 25 21:23 _SUCCESS
-rw-r--r-- 1 saheedyusuf saheedyusuf 8.6M Feb 25 21:23 part-00000-2db1a237-b145-41cc-b951-386590d59af1-c000.snappy.parquet
-rw-r--r-- 1 saheedyusuf saheedyusuf 8.6M Feb 25 21:23 part-00001-2db1a237-b145-41cc-b951-386590d59af1-c000.snappy.parquet
-rw-r--r-- 1 saheedyusuf saheedyusuf 8.6M Feb 25 21:23 part-00002-2db1a237-b145-41cc-b951-386590d59af1-c000.snappy.parquet
-rw-r--r-- 1 saheedyusuf saheedyusuf 8.6M Feb 25 21:23 part-00003-2db1a237-b145-41cc-b951-386590d59af1-c000.snappy.parquet
-rw-r--r-- 1 saheedyusuf saheedyusuf 8.6M Feb 25 21:23 part-00004-2db1a237-b145-41cc-b951-386590d59af1-c000.snappy.parquet
-rw-r--r-- 1 saheedyusuf saheedyusuf 8.6M Feb 25 21:23 part-00005-2db1a237-b145-41cc-b951-386590d59af1-c000.snappy.parquet
-rw-r--r-- 1 saheedyusuf saheedyusuf 8.6M Feb 25 21:23 part-00006-2db1a237-b145-41cc-b951-386590d59af1-c000.snappy.parquet
-rw-r--r-- 1 saheedyusuf saheedyusuf 8.6M Feb 25 21:23 part-0000